# LlamaParse

## Overview

`LlamaParse` 是由 **LlamaIndex** 開發的文件解析服務，專為大語言模型（LLMs）應用而設計。其目的是透過高品質的文件解析強化 RAG（Retrieval-Augmented Generation）等應用的效能。

### ✨ 主要功能

- 支援多種文件格式：PDF、Word、PowerPoint、Excel
- 可透過自然語言指令自訂輸出格式
- 具備進階的表格與圖像抽取功能
- 支援多語言內容
- 提供多種輸出格式（如 JSON、Markdown、Text）

LlamaParse 同時作為獨立 API 與 LlamaCloud 平台的整合服務提供。每位使用者每日可免費處理最多 **1,000 頁**，如需更高容量可升級至付費方案。

> 📢 *目前為公開測試階段（Public Beta），功能持續擴充中。*

- 官方平台：[https://cloud.llamaindex.ai](https://cloud.llamaindex.ai)

---

## 📚 目錄

- [Overview](#overview)
- [Environement Setup](#environment-setup)
- [Data Preparation](#data-preparation)
- [LlamaParse Parameters](#llamaparse-parameters)
- [Simple Parsing](#simple-parsing)
- [Multimodal Model Parsing](#multimodal-model-parsing)
- [Custom Parsing Instructions](#custom-parsing-instructions)

---

## 🔗 延伸閱讀

- [LlamaParse: Using in Python](https://docs.cloud.llamaindex.ai/llamaparse/getting_started/python)
- [LlamaParse: Getting Started](https://docs.cloud.llamaindex.ai/llamaparse/getting_started)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "llama-index-core",
        "llama-parse",
        "llama-index-readers-file",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### API Key Configuration

To use ```LlamaParse``` , you need to [obtain a Llama Cloud API key](https://docs.cloud.llamaindex.ai/llamaparse/getting_started/get_an_api_key).

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LlamaParse",
        "LLAMA_CLOUD_API_KEY": "",
    }
)

Environment variables have been set successfully.


You can alternatively set API keys such as ```OPENAI_API_KEY``` in a ```.env``` file and load them.

**[Note]** This is not necessary if you've already set the required API keys in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
import os
import nest_asyncio

# Allow async
nest_asyncio.apply()

## Data Preparation

In this tutorial, we will use the following pdf file:

- Author: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
- Download Link: [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- File name: "1706.03762v7.pdf"
- File path: "data/1706.03762v7.pdf"

In [6]:
# Download and save sample PDF file to ./data directory
import requests


def download_pdf(url, save_path):
    """
    Downloads a PDF file from the given URL and saves it to the specified path.

    Args:
        url (str): The URL of the PDF file to download.
        save_path (str): The full path (including file name) where the file will be saved.
    """
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an error for bad status codes

        # Save the file to the specified path
        with open(save_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"PDF downloaded and saved to: {save_path}")
    except Exception as e:
        print(f"An error occurred while downloading the file: {e}")


# Configuration for the PDF file
pdf_url = "https://arxiv.org/pdf/1706.03762v7"
file_path = "./data/1706.03762v7.pdf"

# Download the PDF
download_pdf(pdf_url, file_path)

PDF downloaded and saved to: ./data/1706.03762v7.pdf


In [7]:
# Set file path
FILE_PATH = "./data/1706.03762v7.pdf"  # modify to your file path

## 🛠️ LlamaParse Parameters

LlamaParse 提供靈活且功能強大的設定參數，讓開發者可根據需求調整文件解析行為。以下是其核心與進階參數的說明：

---

### 🔑 Key Parameters

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `api_key` | 用於驗證的 LlamaParse API 金鑰 | **必填** |
| `base_url` | LlamaParse API 的基礎網址 | `"DEFAULT_BASE_URL"` |
| `check_interval` | 檢查解析任務狀態的間隔秒數 | `1` |
| `ignore_errors` | 是否在解析錯誤時跳過該檔案 | `True` |
| `max_timeout` | 等待解析任務完成的最大時間（秒） | `2000` |
| `num_workers` | 同時請求的最大數量（1–9） | `4` |
| `result_type` | 輸出格式（如 `"text"`、`"markdown"`、`"json"`、`"structured"`） | `"text"` |
| `show_progress` | 是否顯示進度條 | `True` |
| `split_by_page` | 是否以頁面切割輸出 | `True` |
| `language` | 指定文件語言 | `"en"` |
| `verbose` | 顯示詳細處理過程 | `True` |

---

### ⚙️ Advanced Parameters

#### 📌 Parsing Modes and Enhancements

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `auto_mode` | 自動選擇最佳解析模式 | `False` |
| `auto_mode_trigger_on_image_in_page` | 若頁面含有圖片，啟用進階解析 | `False` |
| `auto_mode_trigger_on_table_in_page` | 若頁面含有表格，啟用進階解析 | `False` |
| `auto_mode_trigger_on_text_in_page` | 若含指定文字時，啟用進階模式 | `None` |
| `auto_mode_trigger_on_regexp_in_page` | 使用正則表達式觸發進階模式 | `None` |
| `premium_mode` | 啟用最進階解析功能（需收費） | `False` |
| `fast_mode` | 使用快速模式（跳過 OCR 與表格重建） | `False` |

---

#### 📄 Content Extraction

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `disable_ocr` | 關閉 OCR，僅提取可選文字 | `False` |
| `disable_image_extraction` | 跳過圖片處理，加速解析 | `False` |
| `extract_charts` | 抽取圖表資料 | `False` |
| `extract_layout` | 提供版面資訊 | `False` |
| `annotate_links` | 註解 URL 與連結位置 | `False` |
| `continuous_mode` | 優化多頁表格抽取品質 | `False` |
| `guess_xlsx_sheet_names` | 嘗試預測 Excel 表單名稱 | `False` |

---

#### 🎨 Output Customization

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `page_separator` | 自定義分頁符號 | `None` |
| `structured_output` | 產生結構化輸出（JSON） | `False` |
| `structured_output_json_schema` | 使用自定義 JSON 結構輸出 | `None` |
| `structured_output_json_schema_name` | 使用預設 schema 名稱輸出 | `None` |
| `parsing_instruction` | 以自然語言提供自訂解析指令 | `""` |

---

#### 🎯 Targeting and Filtering

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `target_pages` | 要解析的頁碼清單（以逗號分隔） | `None` |
| `max_pages` | 限制最多處理的頁數 | `None` |
| `bbox_top`, `bbox_bottom`, `bbox_left`, `bbox_right` | 使用 0–1 範圍指定抽取區域（相對頁面） | `None` |
| `skip_diagonal_text` | 忽略非水平（如斜體）文字 | `False` |

---

#### 🌐 Integration and Webhooks

| 參數名稱 | 說明 | 預設值 |
|----------|------|--------|
| `webhook_url` | 任務完成時要回呼的 URL | `None` |
| `output_s3_path_prefix` | 上傳輸出結果至 S3 的路徑前綴 | `None` |
| `custom_client` | 提供自定義的 HTTPX 客戶端物件 | `None` |
| `invalidate_cache` | 忽略快取並強制重新解析 | `False` |
| `do_not_cache` | 不儲存解析結果快取 | `False` |

---

這些參數提供了極大的彈性，讓使用者可以根據實際文件特性、資源限制與應用需求，最佳化 LlamaParse 的文件解析效能與輸出格式。

## 📄 簡易解析（Simple Parsing）

```LlamaParse``` 的預設用法示範了如何使用其核心功能來解析文件。此模式針對「簡潔性」進行最佳化，非常適合標準文件類型，如 PDF、Word、PowerPoint 和 Excel 等格式。

此模式特別適用於：

- **使用簡單** —— 幾乎不需要額外設定  
- **常見應用場景** —— 基礎的文字內容擷取  
- **快速原型開發** —— 適合測試與小規模應用

---

### ✅ 基本範例

以下是使用 Python 和 `LlamaParse` 解析本機 PDF 文件的簡單範例：

```python
from llama_parse import LlamaParse

# 用你的 API 金鑰初始化 parser
parser = LlamaParse(api_key="your-api-key")

# 解析 PDF 檔案（預設 result_type="text"）
docs = parser.load_data("example.pdf")

# 顯示解析後的文字內容
print(docs[0].text)

In [8]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Configure the LlamaParse instance
parser = LlamaParse(
    result_type="markdown",  # Output format ("text", "markdown", "json", or "structured")
    num_workers=8,
    verbose=True,
    language="en",
    show_progress=True,
)

# Define a file extractor mapping file extensions to parsers
file_extractor = {".pdf": parser}

# Use SimpleDirectoryReader to parse the specified PDF file
documents = SimpleDirectoryReader(
    input_files=[FILE_PATH],  # List of files to process
    file_extractor=file_extractor,
).load_data()

Started parsing the file under job_id a1da411e-8d11-468f-98b9-5c846fccc4a0


In [9]:
# Display the number of documents parsed
len(documents)

15

In [10]:
# Display the first document
documents[0]

Document(id_='801f0227-e66c-424b-b82b-4fefc746ec96', embedding=None, metadata={'file_path': 'data/1706.03762v7.pdf', 'file_name': '1706.03762v7.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2025-01-08', 'last_modified_date': '2025-01-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Provided proper attribution is provided, Google hereby grants permission to reproduce the tables and figures in this paper solely for use in journalistic or scholarly works.\n\n# Attention Is All You Need\n\narXiv:1706.03762v7  [cs.CL]  2 Aug 2023\n\nAshish Vaswani∗                   Noam Shazeer∗                   Niki Parma

### Conversion to LangChain Documents

The parsed documents are converted to the ```LangChain``` document format for further processing.

In [11]:
# Convert LlamaIndex documents to LangChain document format
docs = [doc.to_langchain_format() for doc in documents]

In [12]:
# Display the content of a specific document (e.g., the 6th document)
print(docs[5].page_content)

# Table 1: Maximum path lengths, per-layer complexity and minimum number of sequential operations for different layer types. n is the sequence length, d is the representation dimension, k is the kernel size of convolutions and r the size of the neighborhood in restricted self-attention.

|Layer Type|Complexity per Layer|Sequential Operations|Maximum Path Length|
|---|---|---|---|
|Self-Attention|O(n2 · d)|O(1)|O(1)|
|Recurrent|O(k · n · d)|O(n)|O(logk(n))|
|Convolutional|O(r · n · d)|O(1)|O(n/r)|

# 3.5 Positional Encoding

Since our model contains no recurrence and no convolution, in order for the model to make use of the order of the sequence, we must inject some information about the relative or absolute position of the tokens in the sequence. To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel as the embeddings, so that the two can be summed. There are many choic

In [13]:
# Display metadata of the first document
docs[0].metadata

{'file_path': 'data/1706.03762v7.pdf',
 'file_name': '1706.03762v7.pdf',
 'file_type': 'application/pdf',
 'file_size': 2215244,
 'creation_date': '2025-01-08',
 'last_modified_date': '2025-01-08'}

## 🧠 多模態模型解析（MultiModal Model Parsing）

在 ```LlamaParse``` 中的 **多模態解析** 功能會使用外部 AI 模型處理包含複雜內容的文件。與直接提取純文字不同，此方法會針對每頁截圖，並以「視覺理解」的方式產出結構化輸出。

此方法特別適用於：
- **非標準版面配置的文件**
- **掃描版本的 PDF 或影像型文件**
- **包含圖像、表格、公式等多媒體的內容**

---

### 🔑 主要特點：

- 📸 **視覺化處理**：不是讀取原始文字，而是對每一頁進行截圖再送至模型解析
- 🧠 **AI 模型整合**：支援像是 ```OpenAI GPT-4o``` 等先進的多模態模型
- 🔧 **可自定義模型與 API 金鑰**：可選擇使用哪個供應商的模型，並傳入自己的金鑰

---

### 🧬 執行流程：

1. **生成截圖**：針對文件每一頁生成頁面截圖
2. **傳送至模型**：將截圖連同指令傳入指定的多模態模型（例如 GPT-4o）
3. **彙整結果**：模型會回傳該頁的文字內容（例如 Markdown 格式），最後整合成完整輸出

---

### ⚙️ 重要參數設定

| 參數名稱 | 說明 | 範例值 |
|----------|------|--------|
| ```use_vendor_multimodal_model``` | 是否啟用外部供應商的多模態模型（設為 `True` 代表啟用） | `True` |
| ```vendor_multimodal_model_name``` | 指定要使用的多模態模型名稱（例如 GPT-4o） | `"openai-gpt4o"` |
| ```vendor_multimodal_api_key``` | 指定多模態模型的 API 金鑰（從環境變數讀取） | `"OPENAI_API_KEY"` |
| ```result_type``` | 回傳的格式（可為 `"markdown"`、`"text"` 等） | `"markdown"` |
| ```language``` | 文件語言（影響模型理解與解析） | `"en"` |
| ```skip_diagonal_text``` | 是否跳過傾斜的文字 | `True` |
| ```page_separator``` | 自訂分頁符號（可為 `None`） | `None` |

---

### 📌 注意事項

- 此模式屬於進階模式，需具備額外的外部 API 權限與使用額度（如 GPT-4o）。
- 適合處理高度格式化的文件，如履歷表、論文、金融報表等。

---

此功能結合視覺分析與語言理解，使文件的解析更貼近人類的閱讀行為。

In [14]:
# Configure the LlamaParse instance to use the vendor multimodal model
multimodal_parser = LlamaParse(
    use_vendor_multimodal_model=True,
    # vendor_multimodal_model_name="openai-gpt4o", # uncomment to specify a model
    # vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"], # uncomment and use your OpenAI API key(uses less llama cloud credits)
    result_type="markdown",
    language="en",
)

In [15]:
# Parse the PDF file using the multimodal parser
parsed_docs = multimodal_parser.load_data(file_path=FILE_PATH)

Started parsing the file under job_id 14e6f376-d585-400a-b3ad-235c9d8070db


In [16]:
# Convert to langchain document format
docs = [doc.to_langchain_format() for doc in parsed_docs]
docs

[Document(id='7328336e-649e-4089-b304-99930e2491b7', metadata={}, page_content='# Attention Is All You Need\n\nAshish Vaswani\\*  \nGoogle Brain  \navaswani@google.com  \n\nNoam Shazeer\\*  \nGoogle Brain  \nnoam@google.com  \n\nNiki Parmar\\*  \nGoogle Research  \nnikip@google.com  \n\nJakob Uszkoreit\\*  \nGoogle Research  \nusz@google.com  \n\nLlion Jones\\*  \nGoogle Research  \nllion@google.com  \n\nAidan N. Gomez\\* †  \nUniversity of Toronto  \naidan@cs.toronto.edu  \n\nŁukasz Kaiser\\*  \nGoogle Brain  \nlukaszkaiser@google.com  \n\nIlia Polosukhin\\* ‡  \nillia.polosukhin@gmail.com  \n\n## Abstract\n\nThe dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convoluti

In [17]:
# Display the content of the first document
print(docs[0].page_content)

# Attention Is All You Need

Ashish Vaswani\*  
Google Brain  
avaswani@google.com  

Noam Shazeer\*  
Google Brain  
noam@google.com  

Niki Parmar\*  
Google Research  
nikip@google.com  

Jakob Uszkoreit\*  
Google Research  
usz@google.com  

Llion Jones\*  
Google Research  
llion@google.com  

Aidan N. Gomez\* †  
University of Toronto  
aidan@cs.toronto.edu  

Łukasz Kaiser\*  
Google Brain  
lukaszkaiser@google.com  

Ilia Polosukhin\* ‡  
illia.polosukhin@gmail.com  

## Abstract

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more par

## Custom Parsing Instructions

You can also specify custom instructions for parsing. This allows you to fine-tune the parser’s behavior to meet specific requirements.

In [18]:
# Configure parsing instruction
parsing_instruction = "You are parsing a research paper. Summarize content of each page in markdown format."

# LlamaParse configuration
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    # vendor_multimodal_model_name="openai-gpt4o",
    # vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="en",
    parsing_instruction=parsing_instruction,
)

# Parse pdf file
parsed_docs = parser.load_data(file_path=FILE_PATH)

# Convert to langchain documents
docs = [doc.to_langchain_format() for doc in parsed_docs]

Started parsing the file under job_id 63423449-c6ff-41ff-9191-da14819b04a0


In [19]:
# Display the content of the first document
print(docs[0].page_content)

**Title:** Attention Is All You Need

**Authors:** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin

**Abstract Summary:**

The paper introduces the Transformer, a novel neural network architecture based solely on attention mechanisms, eliminating the need for recurrence and convolutions. This model is designed for sequence transduction tasks, such as machine translation. The Transformer demonstrates superior performance, achieving 28.4 BLEU on the WMT 2014 English-to-German translation task and setting a new state-of-the-art BLEU score of 41.8 on the WMT 2014 English-to-French task. The model is more efficient, requiring less training time and resources, and generalizes well to other tasks, including English constituency parsing.
